In [2]:
import mne
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset

In [4]:
raw1 = mne.io.read_raw_edf('CHB-MIT/chb01_01.edf')
raw2 = mne.io.read_raw_edf('CHB-MIT/chb01_02.edf')

Extracting EDF parameters from /home/matth/python/SeizureSense/CHB-MIT/chb01_01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/matth/python/SeizureSense/CHB-MIT/chb01_02.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


/tmp/ipykernel_1946/878268199.py:1: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  raw1 = mne.io.read_raw_edf('CHB-MIT/chb01_01.edf')
/tmp/ipykernel_1946/878268199.py:2: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  raw2 = mne.io.read_raw_edf('CHB-MIT/chb01_02.edf')


In [1]:
class CHBData(Dataset):
    def __init__(self, CHB_files, segment_length):
        self.segments = []
        self.labels = []
        for file_path, info in CHB_files:
            processed_data=preprocessing(self, file_path)
            for start, end, label in info:
                segment, lab_el = segment_eeg(start, end, label)
                self.segments.append(segment)
                self.labels.append(lab_el)

    def __len__():
        return len(CHB_files)
    
    def __getitem__(self, i):
        return null

    def segment_eeg(self,start, end, label):
        return null
    
    def preprocessing(self,file_path):
        #loading data: 
        raw = mne.io.read_raw_edf(file_path)
        raw.load_data()
        #proccesing every raw object to remove 60 hz and its multiples:
        eeg_picks = mne.pick_types(raw.info, meg=False, eeg=True)
        freqs = (60,120)
        raw_notch = raw.copy().notch_filter(freqs=freqs, picks=eeg_picks)
        #applying a high pass filter of order 4 with a cutoff frequency of 30 Hz to the data to enhance gamma signal to noise ratio:
        raw_notch.filter(l_freq=30, h_freq=None, fir_design='firwin', filter_length='auto', phase='zero', fir_window='hamming')
        numpy_array=raw_notch.get_data()
        segment_tensor=torch.from_numpy(numpy_array)
        return segment_tensor

NameError: name 'Dataset' is not defined

In [5]:
CHB_files = {'CHB-MIT/chb01_01.edf':[(0, 3600, "interictal")], 'CHB-MIT/chb01_02.edf':[(0, 3600, "interictal")], }